
# Proyecto 1 de Diseño Lógico
## Implementación del algoritmo de Quine-McCluskey 

/

<div class="pull-left">

<center>David Medina Mayorga </center>
<center>Escuela de Ingeniería en Electrónica<center>
<center>Instituto Tecnológico de Costa Rica<center>
<center>Cartago, Costa Rica<center>
<center>david.medina@estudiantec.cr<center>

</div> 
 
<div class="pull-right">
<center>Jimena León Huertas </center>
<center>Escuela de Ingeniería en Computadores<center>
<center>Instituto Tecnológico de Costa Rica<center>
<center>Cartago, Costa Rica<center>
<center>mena.leon@estudiantec.cr<center>.

</div> 
 

### Introducción

La minimización lógica puede convertirse en un proceso muy complicado. En las aplicaciones del diseño lógico se pueden ver impicadas funciones de pocas variables que se pueden analizar y minimizar utilizando métodos visuales como los mapas de Karnaugh, sin embargo, también se pueden encontrar funciones con múltiples variables más complejas que son imposibles de minimizar sin el uso de un algoritmo computacional.

En este proyecto se implementará el algoritmo tabular de Quine-McCluskey que consiste en encontrar todos los implicantes primos de la función booleana y seleccionar un conjunto mínimo de implicantes primos que cubra la función. Este algoritmo es exacto, esto quiere decir que se garantiza una minimización máxima de la función. 

Debido a la necesidad creciente de aumentar la densidad de los chips VLSI, se han descubierto métodos más efectivos para la minimización de funciones lógicas, por ejemplo, el algoritmo Espresso. Este algoritmo es un método heurístico, se reduce el tamaño de memoria y tiempo de ejecución, sin embargo, no garantiza una minimización máxima. A pesar de no minimizar la máximo, se encuentra una buena solución para hasta docenas de variables en un muy corto periodo. El algoritmo Espresso se utilizará para comparar el tiempo de ejecución con la implementación de Quine-McCluskey realizada.

### Código

In [1]:
#Librerias
from pyeda.inter import * 
import string

##### Algoritmo Quine-McCluskey

##### Algoritmo Espresso

In [2]:
#FUNCIONES

"Se construye la dimensión de la columna Y de la tabla de verdad en función de la cantidad de variables escogidas"
def ConstruirY (num_variables): 
    
    Yvar = "0"  
    Yaux = ",0" 
    
    for i in range(1,(2**num_variables),1):
        Yvar = Yvar+Yaux
        
    return Yvar #Se retorna el string base de 2**n ceros


"Se construye la columna Y de la tabla de verdad en función de la suma de minterminos"
def ObtenerYtts (minterminos, Y): #Se recibe una lista de mintérminos y una "máscara" de la columna Y con 2**n ceros
    
    Y6var_lista = Y.split(",") 
    
    minTer = minterminos  
    minTer_int = list(map(int,  minTer))
    Y6var_int_list = list(map(int, Y6var_lista))
    
    for i in minTer_int :
        Y6var_int_list[i] = 1
    
    Y6var_int_list = list(map(str, Y6var_int_list))
    Ytts = "".join(Y6var_int_list)
   
    return Ytts #Se retorna la columna Y de la tabla de verdad 


"Minimización de tablas de verdad con el algoritmo Espresso"
def EspressoMin (num_variables,Ytts): #Se recibe la cantidad de variables y las salidas de la tabla de verdad
    
    X = ttvars('x', num_variables) #Se diferencian las variables de acuerdo a su posición
    fbool = truthtable(X,Ytts) #Se define la función booleana con las variables y la tabla de verdad
    fboolm = espresso_tts(fbool) #Se minimiza la función booleana
    
    return fboolm #Se retorna la función booleana minimizada

In [3]:
#Implementación de Espresso básica en consola 

string_Minterms = input("Suma de minterminos:  ")
minterminos_Lista = string_Minterms.split(",")  # Con esto cada mintermino se mete en una lista

num_variables = int(input("Cantidad de variables: "))

print("\n") 

Y = ConstruirY(num_variables) #Se construye las salidas de la tabla de verdad
print(EspressoMin(num_variables, ObtenerYtts(minterminos_Lista,Y))) #Se implementa la minimización con Espresso

Suma de minterminos:  1,3,4,5,9,11,12,13,14,15
Cantidad de variables: 4


(Or(And(x[0], ~x[2]), And(x[2], x[3]), And(~x[1], x[2])),)


### Referencias

- PyEDA https://pyeda.readthedocs.io/en/latest/
- Felipe Santiago Espinoza. Video demostrativo del algoritmo de Quine McCluskey. URL: https://www.youtube.com/watch?v=DTOzK88Inkk
- Synthesis of Digital Systems - IITD. Two-level Logic Optimisation. URL: https://www.youtube.com/watch?v=isqtQmWpDtg
- F. Wakerly. Diseño Digital: principios y prácticas. Pearson, México, 2001
